#1. 地理空間データ

##  1.2 地理空間データのハンドリング(2)


In [1]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/mount')

Mounted at /content/mount


In [2]:
# ワーキングディレクトリの設定
%cd '/content/mount/MyDrive/SDS'

/content/mount/MyDrive/SDS


In [3]:
# geopandasなどのインストール
!pip install -q geopandas
!pip install -q folium
!pip install -q mapclassify

In [4]:
# 必要なライブラリのインポート
import json
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import plugins
import mapclassify

In [5]:
# 大阪駅の緯度経度(リスト)
osaka_station = [34.702635326817834, 135.49595059753435]
# 地図生成
folium_map = folium.Map(location=osaka_station, zoom_start=10)
# 地図表示
folium_map

In [6]:
# leaflet-providersのサイトにアクセスして"OpenStreetMap.HOT"を選択
# 上部の"Plain JavaScript:"に表示されている情報をコピー＆ペースト
attr = (
    '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, Tiles style by <a href="https://www.hotosm.org/" target="_blank">Humanitarian OpenStreetMap Team</a> hosted by <a href="https://openstreetmap.fr/" target="_blank">OpenStreetMap France</a>'
)
tiles = 'https://{s}.tile.openstreetmap.fr/hot/{z}/{x}/{y}.png'

# 地図生成
folium_map2 = folium.Map(location=osaka_station, zoom_start=10, tiles=tiles, attr=attr)
# 地図表示
folium_map2

In [7]:
# ファイルパスの設定
DISTRICT_PATH = "./Chapter1/data/N03-20220101_27_GML/N03-22_27_220101.geojson"
LAND_PATH = "./Chapter1/data/L01-23_27_GML/L01-23_27.geojson"

In [8]:
# geojsonファイル読込
# ポリゴン（行政区域データ）
gdf_district = gpd.read_file(DISTRICT_PATH) # geojson読込
# ポイント（地価公示データ）
gdf_land = gpd.read_file(LAND_PATH) # geojson読込

In [9]:
# 変換後の空間座標系指定
# JGD2000緯度経度(EPSG4612) → JGD2011緯度経度(EPSG6668)
dst_proj = 6668 # 変換後の座標系のEPSGコードを指定
# 地価公示データのgeopandasデータフレームについて空間座標系を一括変換
# to_crs()関数を用いて、引数で空間座標系のEPSGコードを指定
gdf_land = gdf_land.to_crs(epsg=dst_proj)

In [10]:
# geopandasデータフレームオブジェクトの作成(行政区域データ)
# 0番目のインデックス(行)の削除
gdf_district = gdf_district.drop(index = 0)
# 必要な列のみで再構成
gdf_district = gdf_district[['N03_003', 'N03_004', 'N03_007', 'geometry']]
# 列名の付け直し
gdf_district.columns = ['city1', 'city2', 'd_code', 'geometry']

In [11]:
# geopandasデータフレームオブジェクトの作成(地価公示データ)
# 必要な列のみで再構成
gdf_land = gdf_land[['L01_024', 'L01_022', 'L01_006', 'L01_026', 'L01_027', 'L01_050', 'L01_047', 'geometry']]
# 列名の付け直し
gdf_land.columns = ['address', 'd_code', 'price', '地積', '利用現況', '用途区分', '周辺の土地利用の状況', 'geometry']

In [12]:
# address列にある余計な空白文字を消去
gdf_land['address'] = gdf_land['address'].str.replace('　', '')
# np.log()で対数変換し、geopandasデータフレームに新たな列(log_price)として追加
gdf_land['log_price'] = np.log(gdf_land['price'])

In [13]:
# 地域コードをもとにグループを作る
grouped = gdf_land.groupby('d_code')
# グループ内の項目(列)の平均値を求めて新しいpandasデータフレームを作成
price_mean_df = grouped.mean()
# 新しいデータフレームの'log_price'列を行政区域データフレームに結合
gdf_district = gdf_district.merge(price_mean_df['log_price'], on = 'd_code')

<ipython-input-13-23cd1a0e6379>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  price_mean_df = grouped.mean()


In [14]:
# 行政区域データの確認
gdf_district.head()

,city1,city2,d_code,geometry,log_price
0,大阪市,都島区,27102,"POLYGON ((135.51481 34.72057, 135.51489 34.720...",12.769969
1,大阪市,福島区,27103,"POLYGON ((135.48403 34.69969, 135.48404 34.699...",13.214822
2,大阪市,此花区,27104,"POLYGON ((135.35833 34.62786, 135.35860 34.627...",12.191524
3,大阪市,此花区,27104,"POLYGON ((135.39884 34.65724, 135.39877 34.657...",12.191524
4,大阪市,此花区,27104,"POLYGON ((135.39893 34.65737, 135.39892 34.657...",12.191524


In [15]:
# 地価公示データの確認
gdf_land.head()

,address,d_code,price,地積,利用現況,用途区分,周辺の土地利用の状況,geometry,log_price
0,大阪府大阪市都島区毛馬町５丁目２３番,27102,197000,185,住宅,2中専,住宅、マンション、倉庫等が混在する住宅地域,POINT (135.52471 34.72501),12.190959
1,大阪府大阪市都島区御幸町１丁目９４番２,27102,281000,165,住宅,1住居,一般住宅の中に、共同住宅等が混在する住宅地域,POINT (135.53445 34.71340),12.546110
2,大阪府大阪市都島区都島北通２丁目１５７番,27102,322000,142,住宅,1住居,中小規模住宅が多く、店舗等の混在する住宅地域,POINT (135.53287 34.70975),12.682307
3,大阪府大阪市都島区中野町２丁目９３番,27102,304000,158,住宅,2住居,住宅、マンション、事業所等の混在する住宅地域,POINT (135.52604 34.70063),12.624783
4,大阪府大阪市都島区都島中通２丁目１０１番,27102,308000,166,住宅,1住居,一般住宅のほかにマンションが見られる住宅地域,POINT (135.53295 34.70401),12.637855


In [16]:
# 地価公示データの一部をGeoJSONデータ形式に変換する
json_example = gdf_land.iloc[:2,:].to_json()
# GeoJSONデータの表示
data = json.loads(json_example) # json.load関数で改めてloadする
# ensure_ascii= Falseで日本語表示
print(json.dumps(data, indent=2, ensure_ascii = False))

{
  "type": "FeatureCollection",
  "features": [
    {
      "id": "0",
      "type": "Feature",
      "properties": {
        "address": "大阪府大阪市都島区毛馬町５丁目２３番",
        "d_code": "27102",
        "price": 197000,
        "地積": 185,
        "利用現況": "住宅",
        "用途区分": "2中専",
        "周辺の土地利用の状況": "住宅、マンション、倉庫等が混在する住宅地域",
        "log_price": 12.190959007720126
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          135.524715,
          34.725005
        ]
      }
    },
    {
      "id": "1",
      "type": "Feature",
      "properties": {
        "address": "大阪府大阪市都島区御幸町１丁目９４番２",
        "d_code": "27102",
        "price": 281000,
        "地積": 165,
        "利用現況": "住宅",
        "用途区分": "1住居",
        "周辺の土地利用の状況": "一般住宅の中に、共同住宅等が混在する住宅地域",
        "log_price": 12.546109948315882
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          135.53444611100008,
          34.713398056000074
        ]
      }
    }
  ],
  "crs":

In [17]:
# 大阪府地価公示データの中心緯度経度
lat = gdf_land['geometry'].y.mean()
lng = gdf_land['geometry'].x.mean()
# 地図生成
folium_map3 = folium.Map(location=[lat, lng], zoom_start=12)
# データフレームをGeoJSON形式に変換して地図に追加
folium.GeoJson(gdf_land.to_json()).add_to(folium_map3)
# 地図表示
folium_map3

In [18]:
# 地図生成
folium_map3 = folium.Map(location=[lat, lng], zoom_start=14)
# ツールチップの設定
tooltip = folium.GeoJsonTooltip(fields=['address', 'price'], labels=True)
# ポップアップの設定
popup = folium.GeoJsonPopup(fields=['地積', '利用現況', '用途区分', '周辺の土地利用の状況'], labels=True)
# データフレームをGeoJSON形式に変換して地図に追加
folium.GeoJson(gdf_land.to_json(), tooltip=tooltip, popup=popup).add_to(folium_map3)
# 地図表示
folium_map3

In [19]:
# explore関数による地図表示(1)
gdf_district.explore()

Output hidden; open in https://colab.research.google.com to view.

In [20]:
# explore関数による地図表示(2)
gdf_district.explore(column='log_price', name='Log Price(mean)', cmap='jet')

Output hidden; open in https://colab.research.google.com to view.

In [21]:
# explore関数による地図表示(3)
map = gdf_district.explore(column='log_price', name='Log Price(mean)', cmap='jet')
folium.LayerControl().add_to(map)  # レイヤのオンオフをコントロール
map

Output hidden; open in https://colab.research.google.com to view.

In [22]:
# explore関数による地図表示(4)
map = gdf_district.explore(column='log_price', name='Log Price(mean)', cmap='jet')
map = gdf_land.explore(m=map, column='log_price', name='Log Price', cmap='jet', legend=False)  #引数'm=map'としてレイヤを追加
folium.LayerControl().add_to(map)
map

Output hidden; open in https://colab.research.google.com to view.

In [23]:
# explore関数による地図表示(5)

# ヒートマップ作成用のデータリストの作成(緯度・経度・住宅価格)
lat_list = gdf_land['geometry'].y.values  # 緯度はgeometryの2番目の値(y)であることに注意
lng_list = gdf_land['geometry'].x.values  # 経度はgeometryの1番目の値(x)であることに注意
log_price_list = gdf_land['log_price'].values  # '.values'でnp.ndarray型に変換しなければならない
HeatMapData = [[lat, lng, log_price] for lat, lng, log_price in zip(lat_list, lng_list, log_price_list)]

# 地図生成
map = folium.Map(location=[lat, lng], zoom_start=10)
# ヒートマップの表示
plugins.HeatMap(data=HeatMapData, radius=5, blur=4, name='Heat Map').add_to(map)  # dataに入力するのはnp.ndarray型

# これまで作成したレイヤも重ねる
map = gdf_district.explore(m=map, column='log_price', name='Log Price(mean)', cmap='jet')
map = gdf_land.explore(m=map, column='log_price', name='Log Price', cmap='jet', legend=False)  #引数'm=map'としてレイヤを追加
folium.LayerControl().add_to(map)
map

Output hidden; open in https://colab.research.google.com to view.

In [24]:
# HTMLで保存
map.save('./Chapter1/figures/Osaka_LandPriceMap.html')